In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR = r'/content/drive/MyDrive/EMotion detection/train - Copy'
TEST_DIR = r'/content/drive/MyDrive/EMotion detection/test - Copy'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [4]:
train = pd.DataFrame()
if not os.path.exists(TRAIN_DIR):
    raise FileNotFoundError(f"The specified directory '{TRAIN_DIR}' does not exist.")
else:
    train['image'], train['label'] = createdataframe(TRAIN_DIR)


surprise completed
neutral completed
sad completed
happy completed
angry completed
fear completed
disgust completed


In [5]:
print(train)

                                                  image     label
0     /content/drive/MyDrive/EMotion detection/train...  surprise
1     /content/drive/MyDrive/EMotion detection/train...  surprise
2     /content/drive/MyDrive/EMotion detection/train...  surprise
3     /content/drive/MyDrive/EMotion detection/train...  surprise
4     /content/drive/MyDrive/EMotion detection/train...  surprise
...                                                 ...       ...
2873  /content/drive/MyDrive/EMotion detection/train...   disgust
2874  /content/drive/MyDrive/EMotion detection/train...   disgust
2875  /content/drive/MyDrive/EMotion detection/train...   disgust
2876  /content/drive/MyDrive/EMotion detection/train...   disgust
2877  /content/drive/MyDrive/EMotion detection/train...   disgust

[2878 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

fear completed
disgust completed
sad completed
angry completed
happy completed
neutral completed
surprise completed


In [7]:
print(test)
print(test['image'])

                                                  image     label
0     /content/drive/MyDrive/EMotion detection/test ...      fear
1     /content/drive/MyDrive/EMotion detection/test ...      fear
2     /content/drive/MyDrive/EMotion detection/test ...      fear
3     /content/drive/MyDrive/EMotion detection/test ...      fear
4     /content/drive/MyDrive/EMotion detection/test ...      fear
...                                                 ...       ...
2506  /content/drive/MyDrive/EMotion detection/test ...  surprise
2507  /content/drive/MyDrive/EMotion detection/test ...  surprise
2508  /content/drive/MyDrive/EMotion detection/test ...  surprise
2509  /content/drive/MyDrive/EMotion detection/test ...  surprise
2510  /content/drive/MyDrive/EMotion detection/test ...  surprise

[2511 rows x 2 columns]
0       /content/drive/MyDrive/EMotion detection/test ...
1       /content/drive/MyDrive/EMotion detection/test ...
2       /content/drive/MyDrive/EMotion detection/test ...
3       /

In [8]:
from tqdm.notebook import tqdm

In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [10]:
train_features = extract_features(train['image'])

  0%|          | 0/2878 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [11]:
test_features = extract_features(test['image'])

  0%|          | 0/2511 [00:00<?, ?it/s]

In [12]:
x_train = train_features/255.0
x_test = test_features/255.0

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [16]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [17]:
model = Sequential()
# convolutional layers
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))


In [18]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [19]:
from keras.optimizers import Adam

In [20]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 50, validation_data = (x_test,y_test))

Epoch 1/50
23/23 [==============================] - 10s 63ms/step - loss: 1.9528 - accuracy: 0.1372 - val_loss: 1.9495 - val_accuracy: 0.0442
Epoch 2/50
23/23 [==============================] - 0s 19ms/step - loss: 1.9451 - accuracy: 0.1501 - val_loss: 1.9503 - val_accuracy: 0.0442
Epoch 3/50
23/23 [==============================] - 0s 19ms/step - loss: 1.9447 - accuracy: 0.1508 - val_loss: 1.9504 - val_accuracy: 0.0442
Epoch 4/50
23/23 [==============================] - 0s 21ms/step - loss: 1.9443 - accuracy: 0.1581 - val_loss: 1.9511 - val_accuracy: 0.0442
Epoch 5/50
23/23 [==============================] - 0s 21ms/step - loss: 1.9432 - accuracy: 0.1564 - val_loss: 1.9484 - val_accuracy: 0.1458
Epoch 6/50
23/23 [==============================] - 0s 21ms/step - loss: 1.9425 - accuracy: 0.1567 - val_loss: 1.9491 - val_accuracy: 0.1521
Epoch 7/50
23/23 [==============================] - 0s 21ms/step - loss: 1.9390 - accuracy: 0.1647 - val_loss: 1.9550 - val_accuracy: 0.1545
Epoch 8/50
2

In [22]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
from keras.models import model_from_json

In [26]:
json_file = open("/content/drive/MyDrive/EMotion detection/facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("/content/drive/MyDrive/EMotion detection/facialemotionmodel.h5")

In [27]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [28]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0